In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('Inputs/attacks.csv', encoding='latin')
df.head()


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
#Test 
df.columns = df.columns.to_series().apply(lambda x: x.strip())

In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Vemos la cantidad de valores vacios que tenemos en la tabla.
El ratio es muy elevado en la mayoría de las filas.

In [5]:
#dividimos las filas vacias entre el total de filas para ver el de 0 a 1 como de vaias estan las filas de al tabla.

df.isnull().sum().sort_values().head(24)/25723

Case Number               0.661704
original order            0.754733
Date                      0.755005
Case Number.2             0.755005
Case Number.1             0.755005
href                      0.755005
pdf                       0.755005
href formula              0.755044
Year                      0.755083
Type                      0.755161
Investigator or Source    0.755666
Injury                    0.756094
Country                   0.756949
Name                      0.763169
Area                      0.772694
Fatal (Y/N)               0.775959
Location                  0.775998
Activity                  0.776154
Sex                       0.776970
Age                       0.865062
Species                   0.865335
Time                      0.885394
Unnamed: 23               0.999922
Unnamed: 22               0.999961
dtype: float64

Usaremos las columans 'Year', 'Area', 'Fatal (Y/N)' y 'Activity' para mi hipotesis.


In [6]:
#Choose the columns that I will use for my hypothesis
lista_para_limpiar=df[["Year","Country","Activity","Fatal (Y/N)"]]
lista_para_limpiar.head(520)

,Year,Country,Activity,Fatal (Y/N)
0,2018.0,USA,Paddling,N
1,2018.0,USA,Standing,N
2,2018.0,USA,Surfing,N
3,2018.0,AUSTRALIA,Surfing,N
4,2018.0,MEXICO,Free diving,N
...,...,...,...,...
515,2014.0,USA,Standing,N
516,2014.0,USA,NaN,N
517,2014.0,BAHAMAS,Spearfishing,N
518,2014.0,USA,Swimming,N


In [7]:
lista_para_limpiar.isnull().sum().sort_values().head(24)/25723

Year           0.755083
Country        0.756949
Fatal (Y/N)    0.775959
Activity       0.776154
dtype: float64

In [8]:
#Con el método "dropna" vamos a filtrar los valores del df "lista_para_limpiar" para dejar solo aquellos NO nulos.
#Usamos "how='all'" para eliminar  las filas o columnas en las que todos sus elementos sean nulos.
lista_para_limpiar=lista_para_limpiar.dropna(axis=0, how='all')


In [9]:
# Ya no tenemos ningún valor nulo.
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Activity       544
Fatal (Y/N)    539
Country         50
Year             2
dtype: int64

In [10]:
lista_para_limpiar

,Year,Country,Activity,Fatal (Y/N)
0,2018.0,USA,Paddling,N
1,2018.0,USA,Standing,N
2,2018.0,USA,Surfing,N
3,2018.0,AUSTRALIA,Surfing,N
4,2018.0,MEXICO,Free diving,N
...,...,...,...,...
6297,0.0,AUSTRALIA,Diving,Y
6298,0.0,AUSTRALIA,Pearl diving,Y
6299,0.0,USA,Swimming,Y
6300,0.0,PANAMA,NaN,Y


In [11]:
#Eliminamos las filas con el año menor a 1543:
lista_para_limpiar = lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar['Year']<=1963].index)
lista_para_limpiar

,Year,Country,Activity,Fatal (Y/N)
0,2018.0,USA,Paddling,N
1,2018.0,USA,Standing,N
2,2018.0,USA,Surfing,N
3,2018.0,AUSTRALIA,Surfing,N
4,2018.0,MEXICO,Free diving,N
...,...,...,...,...
3712,1964.0,AUSTRALIA,NaN,UNKNOWN
3713,1964.0,SOUTH AFRICA,Diving for sinkers,N
3714,1964.0,NEW ZEALAND,Snorkeling,NaN
3888,1971.0,TURKEY,Swimming,N


In [12]:
lista_para_limpiar["Year"].value_counts().head()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
Name: Year, dtype: int64

In [13]:
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Fatal (Y/N)    300
Activity       243
Country         11
Year             2
dtype: int64

In [14]:
#Usamos el método fillna para sustituir los valores nulos de la columna 'Year'. Si no lo usamos cuando cambiamos 'Year' aparece el error "Cannot convert non-finite values (NA or inf) to integer"
lista_para_limpiar['Year'] = lista_para_limpiar['Year'].fillna(0)
lista_para_limpiar['Year']

0       2018.0
1       2018.0
2       2018.0
3       2018.0
4       2018.0
         ...  
3712    1964.0
3713    1964.0
3714    1964.0
3888    1971.0
6079       0.0
Name: Year, Length: 3717, dtype: float64

In [15]:
#Le quitamos el '.0' a los años.
lista_para_limpiar.Year = lista_para_limpiar.Year.astype(int)
lista_para_limpiar

,Year,Country,Activity,Fatal (Y/N)
0,2018,USA,Paddling,N
1,2018,USA,Standing,N
2,2018,USA,Surfing,N
3,2018,AUSTRALIA,Surfing,N
4,2018,MEXICO,Free diving,N
...,...,...,...,...
3712,1964,AUSTRALIA,NaN,UNKNOWN
3713,1964,SOUTH AFRICA,Diving for sinkers,N
3714,1964,NEW ZEALAND,Snorkeling,NaN
3888,1971,TURKEY,Swimming,N


In [16]:
lista_para_limpiar = lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar['Activity']==0].index)

In [17]:
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Fatal (Y/N)    300
Activity       243
Country         11
Year             0
dtype: int64

In [18]:
lista_para_limpiar.shape

(3717, 4)

In [19]:
lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar.Activity.isnull()].index, inplace = True)

In [20]:
lista_para_limpiar.shape

(3474, 4)

In [21]:
#Quitamos las filas con Nulos de la columna "Activity"
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Fatal (Y/N)    246
Country          8
Activity         0
Year             0
dtype: int64

In [22]:
lista_para_limpiar['Fatal (Y/N)'] = lista_para_limpiar['Fatal (Y/N)'].fillna(0)

In [23]:
#Quitamos las filas con Nulos de la columna "Fatal (Y/N)"
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Country        8
Fatal (Y/N)    0
Activity       0
Year           0
dtype: int64

In [24]:
lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar.Country.isnull()].index, inplace = True)

In [25]:
#Ahora no existen  filas con nulos en las 4 columnas con las que vamos a trabajar.
lista_para_limpiar.isnull().sum().sort_values(ascending=False).head(20)

Fatal (Y/N)    0
Activity       0
Country        0
Year           0
dtype: int64

In [26]:
lista_para_limpiar.shape

(3466, 4)

In [27]:
lista_para_limpiar = lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar['Activity']!="Swimming"].index)


In [28]:
lista_para_limpiar.shape

(469, 4)

In [29]:
lista_para_limpiar = lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar['Fatal (Y/N)']=="UNKNOWN"].index)
lista_para_limpiar = lista_para_limpiar.drop(lista_para_limpiar[lista_para_limpiar['Fatal (Y/N)']==0].index)


In [30]:
def result_of_the_attack(i):
    if i in ["Y", "y"]:
        return "Yes"
    elif i in ["N"," N"]:
        return "No"    

In [31]:

lista_para_limpiar["Fatal (Y/N)"]= lista_para_limpiar["Fatal (Y/N)"].apply(result_of_the_attack)

In [32]:
lista_para_limpiar

,Year,Country,Activity,Fatal (Y/N)
6,2018,BRAZIL,Swimming,Yes
12,2018,USA,Swimming,No
17,2018,THAILAND,Swimming,No
26,2018,THAILAND,Swimming,No
33,2018,BAHAMAS,Swimming,No
...,...,...,...,...
3699,1964,NEW ZEALAND,Swimming,Yes
3703,1964,COSTA RICA,Swimming,Yes
3711,1964,FIJI,Swimming,No
3888,1971,TURKEY,Swimming,No


In [33]:
lista_para_limpiar=lista_para_limpiar.rename(columns = {'Fatal (Y/N)':'Fatal'})

In [34]:
lista_para_limpiar

,Year,Country,Activity,Fatal
6,2018,BRAZIL,Swimming,Yes
12,2018,USA,Swimming,No
17,2018,THAILAND,Swimming,No
26,2018,THAILAND,Swimming,No
33,2018,BAHAMAS,Swimming,No
...,...,...,...,...
3699,1964,NEW ZEALAND,Swimming,Yes
3703,1964,COSTA RICA,Swimming,Yes
3711,1964,FIJI,Swimming,No
3888,1971,TURKEY,Swimming,No


In [35]:
decades = lista_para_limpiar.groupby(["Country","Activity", "Fatal"]).agg({"Fatal":"count"})
decades.head()

Fatal
Country   Activity Fatal       
AUSTRALIA Swimming No        40
                   Yes        8
BAHAMAS   Swimming No         2
BRAZIL    Swimming No        10
                   Yes       18

In [36]:
by_country=lista_para_limpiar.sort_values('Fatal')

In [37]:
by_country.head(5)

,Year,Country,Activity,Fatal
1501,2006,AUSTRALIA,Swimming,No
1992,2000,AUSTRALIA,Swimming,No
1990,2000,USA,Swimming,No
1978,2001,CUBA,Swimming,No
3416,1969,AUSTRALIA,Swimming,No
